In [ ]:
%pip install openai

In [ ]:
# %%writefile -a ./.env

# AZURE_OPENAI_API_KEY=
# AZURE_OPENAI_ENDPOINT=

In [9]:
import os
from dotenv import load_dotenv
load_dotenv()

from openai import AzureOpenAI
import base64

# Initialize Azure OpenAI client
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version="2024-05-01-preview",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
)

# Function to encode image to base64
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

# Path to your image
image_path = "OpenAI_demo_files/the_last_supper.jpeg"
base64_image = encode_image(image_path)

# Create the message with both text and image
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": "Counting the number of people in the image"
            },
            {
                "type": "image_url",
                "image_url": {
                    "url": f"data:image/jpeg;base64,{base64_image}"
                }
            }
        ]
    }
]

# Send request to GPT-4V
response = client.chat.completions.create(
    model="gpt-4o", 
    messages=messages,
    max_tokens=300
)

# Print the response
print(response.choices[0].message.content)

There are 12 people in the image excluding the blurred face.


In [3]:
from openai import AzureOpenAI
from dotenv import load_dotenv
load_dotenv()

import os
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
    api_version="2024-05-01-preview",
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    )

assistant = client.beta.assistants.create(
  name="Financial Analyst Assistant",
  instructions="You are an expert financial analyst. Use your knowledge base to answer questions about audited financial statements.",
  model="gpt-4o",
  tools=[{"type": "file_search"}],
)

In [10]:

# Create a vector store called "Financial Statements"
vector_store = client.beta.vector_stores.create(name="Financial Statements")
 
# Ready the files for upload to OpenAI
file_paths = ["OpenAI_demo_files/financial_results.pdf"]
file_streams = [open(path, "rb") for path in file_paths]
 
# Use the upload and poll SDK helper to upload the files, add them to the vector store,
# and poll the status of the file batch for completion.
file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
  vector_store_id=vector_store.id, files=file_streams
)
 
assistant = client.beta.assistants.update(
  assistant_id=assistant.id,
  tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
)

# You can print the status and the file counts of the batch to see the result of this operation.
print(file_batch.status)
print(file_batch.file_counts)

completed
FileCounts(cancelled=0, completed=1, failed=0, in_progress=0, total=1)


In [11]:
thread = client.beta.threads.create()

# Add a user question to the thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="What are the major source of revenue for the company?"
)

# Run the thread and poll for the result
run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions="Please answer based on files provided",
)

if run.status == "completed":
    messages = client.beta.threads.messages.list(thread_id=thread.id)
    for message in messages.data[::-1]:
        print(f"{message.role}: {'/n'.join([x.text.value for x in message.content])}")

user: What are the major source of revenue for the company?
assistant: The major sources of revenue for the company are:

1. **Subscription Revenue**: This is the primary source of revenue, significantly higher than other sources.
2. **Product Revenue**: This includes sales of software and other products.
3. **Services and Other Revenue**: This includes various services provided by the company.

For the three months ended on May 31, 2024, subscription revenue was $5.06 billion, product revenue was $104 million, and services and other revenue was $145 million【4:0†source】.


In [12]:
message.content[0].text.annotations

[FileCitationAnnotation(end_index=505, file_citation=FileCitation(file_id='assistant-z76bpb4Cn93PzueCFnfZuNnY'), start_index=493, text='【4:0†source】', type='file_citation'),
 FileCitationAnnotation(end_index=505, file_citation=FileCitation(file_id='assistant-z76bpb4Cn93PzueCFnfZuNnY'), start_index=493, text='【4:0†source】', type='file_citation')]

In [13]:

# Add a user question to the thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Can you elaborate on source 2?"
)

# Run the thread and poll for the result
run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions="Please answer based on files provided",
)

if run.status == "completed":
    messages = client.beta.threads.messages.list(thread_id=thread.id)
    for message in messages.data[::-1]:
        print(f"{message.role}: {'/n'.join([x.text.value for x in message.content])}")

user: What are the major source of revenue for the company?
assistant: The major sources of revenue for the company are:

1. **Subscription Revenue**: This is the primary source of revenue, significantly higher than other sources.
2. **Product Revenue**: This includes sales of software and other products.
3. **Services and Other Revenue**: This includes various services provided by the company.

For the three months ended on May 31, 2024, subscription revenue was $5.06 billion, product revenue was $104 million, and services and other revenue was $145 million【4:0†source】.
user: Can you elaborate on source 2?
assistant: ### Product Revenue

**Product revenue** includes revenue from sales of software and other products. For the three months ended May 31, 2024, product revenue was $104 million, which is a decrease from $130 million for the same period in the previous year. The decline in product revenue is also evident over the six months period, with the revenue decreasing from $250 milli

In [20]:
run.usage

Usage(completion_tokens=179, prompt_tokens=10869, total_tokens=11048)